In [82]:
import os
import chardet
import re
import ast
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2Tokenizer
from pandarallel import pandarallel

pd.set_option("display.max_colwidth", None)
pandarallel.initialize(progress_bar=True, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [95]:
DATASET_FILENAME = "movie_conversations.pkl"
MAX_TOKEN_LENGTH = 1024
dataset_generated = False

if os.path.exists(DATASET_FILENAME):
    dataset_generated = True

In [84]:
if not dataset_generated:
    # Detect encoding of the movie lines file
    cwd = os.getcwd()

    with open(cwd + "/movie_lines/movie_lines.txt", "rb") as f:
        result = chardet.detect(f.read())
        movie_lines_encoding = result["encoding"]

    with open(cwd + "/movie_lines/movie_conversations.txt", "rb") as f:
        result = chardet.detect(f.read())
        movie_conversations_encoding = result["encoding"]

    print(movie_lines_encoding)
    print(movie_conversations_encoding)

In [85]:
if not dataset_generated:
    # Collect individual movie lines
    with open(cwd + "/movie_lines/movie_lines.txt", "r", encoding=movie_lines_encoding) as f:
        content = f.read()

    lines = content.split("\n")

    # Print first 5 lines and verify length is correct
    print(lines[:5])

    # Remove last element of lines because its an empty string
    lines = lines[:-1]
    print(len(lines))


In [86]:
if not dataset_generated:
    # Initialize containers for values to put in dataframe
    line_numbers = []
    character_ids = []
    character_lines = []

    for line in lines:
        # Split on whitespace
        split = line.split(" ")
        line_numbers.append(split[0])
        character_ids.append(split[2])

        # Extract the text after the last "+" character
        l = re.split(r'\+\s+(?=[^+]*$)', line)[-1]
        character_lines.append(l)


    # Create dataframe from extracted values
    movie_lines = pd.DataFrame(list(zip(line_numbers, character_ids, character_lines)), columns=["line_id", "character_id", "line"])
    display(movie_lines.head(10))
    movie_lines.info()

In [87]:
if not dataset_generated:
    # Collect movie conversation lists
    with open(cwd + "/movie_lines/movie_conversations.txt", "r", encoding=movie_conversations_encoding) as f:
        content = f.read()

    lines = content.split("\n")

    # Print first 5 lines and verify length is correct
    print(lines[:5])

    # Remove last element of lines because its an empty string
    lines = lines[:-1]
    print(len(lines))

In [88]:
if not dataset_generated:
    # Initialize containers for values to put in dataframe
    speaker1_ids = []
    speaker2_ids = []
    conversation_lines = []

    for line in lines:
        # Split on whitespace
        split = line.split(" ")
        speaker1_ids.append(split[0])
        speaker2_ids.append(split[2])

        # Extract the text after the last "+" character
        l = re.split(r'\+\s+(?=[^+]*$)', line)[-1]
        l = ast.literal_eval(l)
        conversation_lines.append(l)


    # Create dataframe from extracted values
    movie_conversations = pd.DataFrame(list(zip(speaker1_ids, speaker2_ids, conversation_lines)), columns=["speaker1_id", "speaker2_id", "conversation_lines"])
    display(movie_conversations.head())
    movie_conversations.info()

In [113]:
display(movie_lines.head())
movie_conversations.head()

,line_id,character_id,line
0,L1045,u0,They do not!
1,L1044,u2,They do to!
2,L985,u0,I hope so.
3,L984,u2,She okay?
4,L925,u0,Let's go.


,speaker1_id,speaker2_id,conversation_lines,conversation,estimated_token_count
0,u0,u2,"[L194, L195, L196, L197]","<USER>: Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. \n<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. \n<USER>: Not the hacking and gagging and spitting part. Please. \n<AGENT>: Okay... then how 'bout we try out some French cuisine. Saturday? Night? \n",59
1,u0,u2,"[L198, L199]",<USER>: You're asking me out. That's so cute. What's your name again? \n<AGENT>: Forget it. \n,15
2,u0,u2,"[L200, L201, L202, L203]","<USER>: No, no, it's my fault -- we didn't have a proper introduction --- \n<AGENT>: Cameron. \n<USER>: The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does. \n<AGENT>: Seems like she could get a date easy enough... \n",51
3,u0,u2,"[L204, L205, L206]","<AGENT>: Why? \n<USER>: Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something. \n<AGENT>: That's a shame. \n",32
4,u0,u2,"[L207, L208]","<USER>: Gosh, if only we could find Kat a boyfriend... \n<AGENT>: Let me see what I can do. \n",18


In [89]:
# Function for turning movie lines into multi-turn conversations for training
def create_conversation_turns(row):
    speaker1 = row["speaker1_id"]
    conversation_list = row["conversation_lines"]
    convo = ""
    # For each line, add it to the conversation with a role label
    for line_id in conversation_list:
        movie_line = movie_lines[movie_lines["line_id"] == line_id]
        text = movie_line["line"].values[0]

        if movie_line["character_id"].values[0] == speaker1:
            convo += f"<USER>: {text} \n"
        else:
            convo += f"<AGENT>: {text} \n"
        
    return convo

In [90]:
if not dataset_generated:
    # Try it on a sample for testing
    # sample = movie_conversations.iloc[:2]
    # sample["conversation"] = sample.apply(create_conversation_turns, axis=1)
    # display(sample)

    # Call create_conversation_turns on every row of the dataframe
    movie_conversations["conversation"] = movie_conversations.parallel_apply(create_conversation_turns, axis=1)

In [103]:
if not dataset_generated:
    with open(DATASET_FILENAME, "wb") as f:
        pickle.dump(movie_conversations, f)
else:
    with open(DATASET_FILENAME, "rb") as f:
        movie_conversations = pickle.load(f)

In [104]:
display(movie_conversations.head())
print(movie_conversations["conversation"].head(1).values[0])

,speaker1_id,speaker2_id,conversation_lines,conversation
0,u0,u2,"[L194, L195, L196, L197]","<USER>: Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again. \n<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. \n<USER>: Not the hacking and gagging and spitting part. Please. \n<AGENT>: Okay... then how 'bout we try out some French cuisine. Saturday? Night? \n"
1,u0,u2,"[L198, L199]",<USER>: You're asking me out. That's so cute. What's your name again? \n<AGENT>: Forget it. \n
2,u0,u2,"[L200, L201, L202, L203]","<USER>: No, no, it's my fault -- we didn't have a proper introduction --- \n<AGENT>: Cameron. \n<USER>: The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser. My sister. I can't date until she does. \n<AGENT>: Seems like she could get a date easy enough... \n"
3,u0,u2,"[L204, L205, L206]","<AGENT>: Why? \n<USER>: Unsolved mystery. She used to be really popular when she started high school, then it was just like she got sick of it or something. \n<AGENT>: That's a shame. \n"
4,u0,u2,"[L207, L208]","<USER>: Gosh, if only we could find Kat a boyfriend... \n<AGENT>: Let me see what I can do. \n"


<USER>: Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again. 
<AGENT>: Well, I thought we'd start with pronunciation, if that's okay with you. 
<USER>: Not the hacking and gagging and spitting part.  Please. 
<AGENT>: Okay... then how 'bout we try out some French cuisine.  Saturday?  Night? 



In [105]:
# Estimate the token count for each conversation so we can split the ones that are too long
movie_conversations["estimated_token_count"] = movie_conversations["conversation"].apply(lambda x: len(x.split()))

In [109]:
# Get the conversation entries that are over 80% of the token limit (80% to be conservative since it's an estimate)
movie_conversations.head()
over_token_limit = movie_conversations[movie_conversations["estimated_token_count"] >= MAX_TOKEN_LENGTH*0.7]
display(over_token_limit)

speaker1_id speaker2_id  \
11348       u1240       u1243   
19670       u2138       u2147   
24318       u2617       u2618   
42477       u4522       u4525   
45571       u4857       u4866   
70355       u7663       u7666   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      conversation_lines  \
11348                                                                                                                                                                                                                                                                                                                             [L267558, L267559, L267560, L267561, L267562, L267563, L267564, L267565, L267566, L267567, L267568, L267569, L267570, L267571, L267572, L267573, L267574, L267575, L267576, L267577, L267578, L267579, L267580, L267581, L267582, L267583, L267584, L267585, L267586, L267587, L267588, L267589, L267590, L267591, L267592, L267593, L267594, L267595, L267596, L267597, L267598, L267599, L267600, L267601, L267602, L267603, L267604, L267605, L267606, L267607, L267608, L267609, L267610, L267611]   
19670                                                                                                                                                                                                                                                                                                                                      [L393396, L393397, L393398, L393399, L393400, L393401, L393402, L393403, L393404, L393405, L393406, L393407, L393408, L393409, L393410, L393411, L393412, L393413, L393414, L393415, L393416, L393417, L393418, L393419, L393420, L393421, L393422, L393423, L393424, L393425, L393426, L393427, L393428, L393429, L393430, L393431, L393432, L393433, L393434, L393435, L393436, L393437, L393438, L393439, L393440, L393441, L393442, L393443, L393444, L393445, L393446, L393447, L393448]   
24318                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [L465557, L465558, L465559, L465560, L465561, L465562, L465563, L465564, L465565, L465566, L465567, L465568, L465569, L465570, L465571, L465572, L465573, L465574, L465575, L465576, L465577, L465578, L465579, L465580, L465581, L465582, L465583, L465584, L465585, L465586, L465587, L465588, L465589, L465590, L465591, L465592, L465593, L465594, L465595]   
42477  [L135360, L135361, L135362, L135363, L135364, L135365, L135366, L135367, L135368, L135369, L135370, L135371, L135372, L135373, L135374, L135375, L135376, L135377, L135378, L135379, L135380, L135381, L135382, L135383, L135384, L135385, L135386, L135387, L135388, L135389, L135390, L135391, L135392, L135393, L135394, L135395, L135396, L135397, L135398, L135399, L135400, L135401, L135402, L135403, L135404, L135405, L135406, L135407, L135408, L135409, L135410, L135411, L135412, L135413, L135414, L135415, L135416, L1354

In [110]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
over_token_limit["actual_token_count"] = over_token_limit["conversation"].apply(lambda x: len(tokenizer.encode(x)))

Token indices sequence length is longer than the specified maximum sequence length for this model (1335 > 1024). Running this sequence through the model will result in indexing errors
/var/folders/8d/zjs9s_1s6njg6wx3_cvk_l5c0000gn/T/ipykernel_78655/2670902946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  over_token_limit["actual_token_count"] = over_token_limit["conversation"].apply(lambda x: len(tokenizer.encode(x)))


In [111]:
over_token_limit

speaker1_id speaker2_id  \
11348       u1240       u1243   
19670       u2138       u2147   
24318       u2617       u2618   
42477       u4522       u4525   
45571       u4857       u4866   
70355       u7663       u7666   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      conversation_lines  \
11348                                                                                                                                                                                                                                                                                                                             [L267558, L267559, L267560, L267561, L267562, L267563, L267564, L267565, L267566, L267567, L267568, L267569, L267570, L267571, L267572, L267573, L267574, L267575, L267576, L267577, L267578, L267579, L267580, L267581, L267582, L267583, L267584, L267585, L267586, L267587, L267588, L267589, L267590, L267591, L267592, L267593, L267594, L267595, L267596, L267597, L267598, L267599, L267600, L267601, L267602, L267603, L267604, L267605, L267606, L267607, L267608, L267609, L267610, L267611]   
19670                                                                                                                                                                                                                                                                                                                                      [L393396, L393397, L393398, L393399, L393400, L393401, L393402, L393403, L393404, L393405, L393406, L393407, L393408, L393409, L393410, L393411, L393412, L393413, L393414, L393415, L393416, L393417, L393418, L393419, L393420, L393421, L393422, L393423, L393424, L393425, L393426, L393427, L393428, L393429, L393430, L393431, L393432, L393433, L393434, L393435, L393436, L393437, L393438, L393439, L393440, L393441, L393442, L393443, L393444, L393445, L393446, L393447, L393448]   
24318                                                                                                                                                                                                                                                                                                                                                                                                                                                                    [L465557, L465558, L465559, L465560, L465561, L465562, L465563, L465564, L465565, L465566, L465567, L465568, L465569, L465570, L465571, L465572, L465573, L465574, L465575, L465576, L465577, L465578, L465579, L465580, L465581, L465582, L465583, L465584, L465585, L465586, L465587, L465588, L465589, L465590, L465591, L465592, L465593, L465594, L465595]   
42477  [L135360, L135361, L135362, L135363, L135364, L135365, L135366, L135367, L135368, L135369, L135370, L135371, L135372, L135373, L135374, L135375, L135376, L135377, L135378, L135379, L135380, L135381, L135382, L135383, L135384, L135385, L135386, L135387, L135388, L135389, L135390, L135391, L135392, L135393, L135394, L135395, L135396, L135397, L135398, L135399, L135400, L135401, L135402, L135403, L135404, L135405, L135406, L135407, L135408, L135409, L135410, L135411, L135412, L135413, L135414, L135415, L135416, L1354